In [1]:
import itertools

#split
def splitLine(line):
	u, fs = line.split('\t')
	fList = fs.split(',')
	return (u, fList)

def checkValLine(line):
	u, fList = line
	return fList[0] != ''

#get friends
def getDirF(line):
	u, fList = line
	dirF = [((u, f), float('-inf')) for f in fList]
	return dirF

def getMutF(line):
	u, fList = line
	mutF = [(mutFPair, 1.) for mutFPair in itertools.permutations(fList, 2)]
	return mutF

#get line
def getWeight(tW, nW):
	return tW + nW

def checkWeight(data):
	pair, weight = data
	return weight > 0

def transform(data):
	pair, weight = data
	u, v = pair
	return (u, (weight, v))

#output
def getRecList(data):
	u, rec = data
	rec.sort(key = lambda rec: (-rec[0], int(rec[1])))
	rec = [v[1] for v in rec[: 10]]
	return (u, rec)

def getResLine(data):
	u , rec = data
	line = u + '\t' + ','.join(v for v in rec)
	return line

def getAllUser(line):
	u, fList = line
	return (u, [])

def getAllRec(tL, nL):
	tL.extend(nL)
	return tL

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
631,application_1548171028063_0175,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
data = sc.textFile('/user/hz333/data/hw1/Q1/soc-LiveJournal1Adj.txt')
line = data.map(splitLine)

#get (u, [friend])
valLine = line.filter(checkValLine)

#get ((u, v), -inf)
dirF = valLine.flatMap(getDirF)
#get ((v, w), 1)
mutF = valLine.flatMap(getMutF)

pair = dirF.union(mutF)

#get (pair, weight)
pair = pair.reduceByKey(getWeight)
pair = pair.filter(checkWeight)

#get (v, (weight, v))
recList = pair.map(transform)

#get (v, [(recommend)])
recList = recList.groupByKey()
recList = recList.mapValues(list)
recList = recList.map(getRecList)

#get (u, [])
userList = line.map(getAllUser)

#get (u, [recommend])
recList = recList.union(userList)
recList = recList.reduceByKey(getAllRec)

#str(data)
resLine = recList.map(getResLine)

resLine = resLine.collect()
sc.parallelize(resLine).repartition(1).saveAsTextFile('/user/hz333/data/hw1/Q1/result')

In [3]:
sc.stop()